In [8]:
import sys
sys.path.append("./src")
from data.data import get_Igakubu_dataset, get_snemid_half_dataset
from data.dataset import DataSetLoader
from utils import Monitor, valid, train
from baseline import RSUNet

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("dark_background")

In [2]:
dst = list()
dst.append((0,0,1))
dst.append((0,1,0))
dst.append((1,0,0))
dst.append((0,0,3))
dst.append((0,3,0))
dst.append((2,0,0))
dst.append((0,0,9))
dst.append((0,9,0))
dst.append((3,0,0))
dst.append((0,0,27))
dst.append((0,27,0))
dst.append((4,0,0))

In [3]:
fov = (32, 160, 160)
num_workers = 2    # musi now
val_freq = 300     # interval between validation(test)

nstart = 0
niter = 50000   #number of train
nvalrun = 20

logdir = "./igaku_logs/finetune_10"
device = 0
nout = len(dst)

In [4]:
#root="/home/neuro/workspace/data/Igakubu/extracted/"
#train_dataset, val_dataset = get_Igakubu_dataset(root, fov=fov, dst=dst)

root   = "/home/neuro/workspace/data/SNEMI3D/original/tif/"
train_dataset, val_dataset = get_snemid_half_dataset(root, fov=fov, dst=dst)

trdl  = DataSetLoader(train_dataset)
valdl = DataSetLoader(val_dataset)

In [5]:
model = RSUNet(12, 4).cuda(device)
#model.load_state_dict(torch.load("./models/model"))
#model.output = nn.Conv3d(24, nout, (1,1,1), bias=True).cuda()

In [6]:
opt = torch.optim.Adam(model.parameters())
loss_fn = F.binary_cross_entropy_with_logits
monitor = Monitor(logdir, False, False, False, False, False)

In [12]:
for i in tqdm(list(range(nstart, niter))):
    a,b = train(model, opt, trdl, loss_fn, device)
    monitor.log_train(i, a, b)  
    if i % val_freq == 0:
        val_stats = valid(model, valdl, loss_fn, device=device)
        trn_stats = valid(model, trdl, loss_fn, device=device)
        monitor.log_val(i, "valid", *val_stats)  
        monitor.log_val(i, "train", *trn_stats)  

  0%|          | 0/50000 [00:00<?, ?it/s]/home/neuro/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/home/neuro/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
  0%|          | 72/50000 [01:41<19:36:49,  1.41s/it]

KeyboardInterrupt: 

In [11]:
from tqdm import tqdm